# Load test gt and pred bboxes

In [ ]:
# Load a .npy file
import numpy as np
import matplotlib.pyplot as plt

filepath = "../val_bboxes/scene0146_00_gt.npy"
gt = np.load(filepath, allow_pickle=True)
# print(gt)
# print(gt[0]['bbox'].shape)

Results from gt_df_results for the scene:

* scene0146_00,otherfurniture,0,True
* scene0146_00,toilet,1,True
* scene0146_00,sink,2,True
* scene0146_00,door,3,False

Classnames-index and class-idx pairs checks out.


In [ ]:
%%sh

pwd

In [ ]:
%%sh
cd ../../../../mnt/data/Datasets/ScanNet_v2/scans/scene0146_00
ls

In [ ]:
from scannet_val import ScannetValDataset

test_dataset = ScannetValDataset(all_classes=True,
                                    num_novel_class=9,
                                    num_points=40000,
                                    augment=False)

'''
ret_dict = {}
        ret_dict['point_clouds'] = point_cloud.astype(np.float32)
        ret_dict['center_label'] = target_bboxes.astype(np.float32)[:, 0:3]
        ret_dict['heading_class_label'] = angle_classes.astype(np.int64)
        ret_dict['heading_residual_label'] = angle_residuals.astype(np.float32)
        ret_dict['size_residual_label'] = size_residuals.astype(np.float32)
        ret_dict['sem_cls_label'] = target_bboxes_semcls.astype(np.int64)
        ret_dict['box_label_mask'] = target_bboxes_mask.astype(np.float32)
        ret_dict['vote_label'] = point_votes.astype(np.float32)
        ret_dict['vote_label_mask'] = point_votes_mask.astype(np.int64)
'''
data = test_dataset[46]
# print(data['point_clouds'].shape)
# print(data.keys())

The keys in your dictionary represent various pieces of data associated with a scene from the ScanNet dataset. Here's a brief explanation of each key:

1. point_clouds: The actual 3D point cloud data, typically in the shape of (N, 3) where N is the number of points, and each point has x, y, z coordinates.
2. center_label: The center coordinates of the ground truth bounding boxes, usually in the shape of (M, 3) where M is the number of bounding boxes.
3. heading_class_label: Class labels for the heading angle of each bounding box.
4. heading_residual_label: Residual values for the heading angles of the bounding boxes.
5. size_residual_label: Residual values for the size dimensions of the bounding boxes.
6. sem_cls_label: Semantic class labels for the bounding boxes.
7. box_label_mask: Mask indicating which bounding boxes are valid (typically a binary array).
8. vote_label: Offsets from the points to their nearest bounding box centers.
9. vote_label_mask: Mask indicating which points have valid vote labels.

In [ ]:
point_clouds = data['point_clouds']

gt_bounding_boxes = []

for box_dict in gt:
    bbox = box_dict['bbox']
    gt_bounding_boxes.append(bbox)
    
# print(len(gt_bounding_boxes))

In [ ]:
import open3d as o3d

# Load the point cloud from a .ply file
pcd = o3d.io.read_point_cloud("../../../../mnt/data/Datasets/ScanNet_v2/scans/scene0146_00/scene0146_00_vh_clean_2.ply")
pcd_labels = o3d.io.read_point_cloud("../../../../mnt/data/Datasets/ScanNet_v2/scans/scene0146_00/scene0146_00_vh_clean_2.labels.ply")

# Extract points and colors
points = np.asarray(pcd.points)
colors = np.asarray(pcd.colors)

# print("Points shape:", points.shape)
# print("Colors shape:", colors.shape)

# Plotting the point clouds only

In [ ]:
o3d.visualization.draw_plotly([pcd])

# Plotting point cloud labels from files in dataset directory

In [ ]:
o3d.visualization.draw_plotly([pcd_labels])

# Plotting bounding boxes from ground-truth dataset object without rotation

In [ ]:
# Function to create a bounding box LineSet from corners
def create_bbox_from_corners(corners):
    lines = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
        [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
        [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical lines
    ]
    colors = [[1, 0, 0] for _ in range(len(lines))]  # Red color for all lines
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(corners),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

# Generate LineSets for all bounding boxes
entities = [pcd]
for corners in gt_bounding_boxes:
    bbox = create_bbox_from_corners(corners)
    entities.append(bbox)

# Visualize the point cloud with bounding boxes using draw_plotly
o3d.visualization.draw_plotly(entities)

In [ ]:
def roty(t):
    """Rotation about the y-axis."""
    c = np.cos(t)
    s = np.sin(t)
    return np.array([[c,  0,  s],
                    [0,  1,  0],
                    [-s, 0,  c]])

def inverse_get_3d_box(corners_3d):
    # Transpose to get (3, 8) shape
    corners_3d = corners_3d.T
    # print("corners_3d shape:", corners_3d.shape)

    # Recover the center by calculating the mean of the corners
    center = np.mean(corners_3d, axis=1)
    # print("center shape:", center.shape)

    # Translate corners back to origin
    corners_3d[0, :] -= center[0]
    corners_3d[1, :] -= center[1]
    corners_3d[2, :] -= center[2]

    # Determine the heading angle by examining the direction of the corners
    x_corners = corners_3d[0, :]
    z_corners = corners_3d[2, :]
    heading_angle = np.arctan2(z_corners[1] - z_corners[0], x_corners[1] - x_corners[0])

    # Rotate corners back to axis-aligned state
    R = roty(-heading_angle)
    aligned_corners = np.dot(R, corners_3d)

    # Determine the box size from the extents of the aligned corners
    l = np.max(aligned_corners[0, :]) - np.min(aligned_corners[0, :])
    w = np.max(aligned_corners[2, :]) - np.min(aligned_corners[2, :])
    h = np.max(aligned_corners[1, :]) - np.min(aligned_corners[1, :])
    
    # modifications:
    box_size = [w, l, h]

    heading_angle = 0
    
    new_center = center.copy()
    new_center[1] = center[2]
    new_center[2] = -center[1]
    
    return box_size, heading_angle, new_center

# Plotting bounding boxes from ScannetValDataset data

In [ ]:
import open3d as o3d
import numpy as np

center_label = data["center_label"]
heading_class_label = data["heading_class_label"]
heading_residual_label = data["heading_residual_label"]
size_residual_label = data["size_residual_label"]
sem_cls_label = data["sem_cls_label"]

# print("No. of objects in scene: ", len(center_label))

classes = ['bathtub', 'bed', 'bookshelf', 'cabinet', 'chair', 'counter', 'curtain', 'desk', 'door', 'otherfurniture',
                      'picture', 'refrigerator', 'showercurtain', 'sink', 'sofa', 'table', 'toilet', 'window']

# Assuming class definitions and sizes
# You will need to define these based on your dataset and model
box_sizes = {'cabinet': np.array([0.76966726, 0.81160211, 0.92573741]),
                       'bed': np.array([1.876858, 1.84255952, 1.19315654]),
                       'chair': np.array([0.61327999, 0.61486087, 0.71827014]),
                       'sofa': np.array([1.39550063, 1.51215451, 0.83443565]),
                       'table': np.array([0.97949596, 1.06751485, 0.63296875]),
                       'door': np.array([0.53166301, 0.59555772, 1.75001483]),
                       'window': np.array([0.96247056, 0.72462326, 1.14818682]),
                       'bookshelf': np.array([0.83221924, 1.04909355, 1.68756634]),
                       'picture': np.array([0.21132214, 0.4206159 , 0.53728459]),
                       'counter': np.array([1.44400728, 1.89708334, 0.26985747]),
                       'desk': np.array([1.02942616, 1.40407966, 0.87554322]),
                       'curtain': np.array([1.37664116, 0.65521793, 1.68131292]),
                       'refrigerator': np.array([0.66508189, 0.71111926, 1.29885307]),
                       'showercurtain': np.array([0.41999174, 0.37906947, 1.75139715]),
                       'toilet': np.array([0.59359559, 0.59124924, 0.73919014]),
                       'sink': np.array([0.50867595, 0.50656087, 0.30136236]),
                       'bathtub': np.array([1.15115265, 1.0546296 , 0.49706794]),
                       'otherfurniture': np.array([0.47535286, 0.49249493, 0.58021168])
                        }


# Function to create a bounding box LineSet from center, heading, and size
def create_bbox(center, heading, size):
    corners = np.array([
        [-size[0]/2, -size[1]/2, -size[2]/2],
        [ size[0]/2, -size[1]/2, -size[2]/2],
        [ size[0]/2,  size[1]/2, -size[2]/2],
        [-size[0]/2,  size[1]/2, -size[2]/2],
        [-size[0]/2, -size[1]/2,  size[2]/2],
        [ size[0]/2, -size[1]/2,  size[2]/2],
        [ size[0]/2,  size[1]/2,  size[2]/2],
        [-size[0]/2,  size[1]/2,  size[2]/2],
    ])
    
    # Rotate the corners according to the heading
    cos_theta = np.cos(heading)
    sin_theta = np.sin(heading)
    rotation_matrix = np.array([
        [1, 0, 0],
        [0, cos_theta, -sin_theta],
        [0, sin_theta, cos_theta]
    ])
    rotated_corners = np.dot(corners, rotation_matrix.T)
    
    # Translate corners to the center
    corners =  center - rotated_corners
    
    # Create LineSet
    lines = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
        [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
        [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical lines
    ]
    colors = [[1, 0, 0] for _ in range(len(lines))]  # Red color for all lines
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(corners),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    return line_set

# Generate LineSets for all bounding boxes
entities = [pcd]
for i in range(len(gt_bounding_boxes)):
    center = center_label[i]
    heading = heading_class_label[i] * (2 * np.pi / 12) + heading_residual_label[i]
    classname = classes[sem_cls_label[i]]
    size = box_sizes[classname] + size_residual_label[i]
    bbox = create_bbox(center, heading, size)
    entities.append(bbox)
    
    print(f"Bounding Box {gt[i]['gt_bbox_index']+1}:")
    print(f"Class {classes[gt[i]['class']]}:")
    print(f"Center: {center}")


# Visualize the point cloud with bounding boxes using draw_plotly
o3d.visualization.draw_plotly(entities)


# Function to plot both GT and Pred BBoxes along with Point Cloud:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scannet_val import ScannetValDataset
import open3d as o3d
import os

classes = ['bathtub', 'bed', 'bookshelf', 'cabinet', 'chair', 'counter', 'curtain', 'desk', 'door', 'otherfurniture',
                      'picture', 'refrigerator', 'showercurtain', 'sink', 'sofa', 'table', 'toilet', 'window']

box_sizes = {'cabinet': np.array([0.76966726, 0.81160211, 0.92573741]),
                       'bed': np.array([1.876858, 1.84255952, 1.19315654]),
                       'chair': np.array([0.61327999, 0.61486087, 0.71827014]),
                       'sofa': np.array([1.39550063, 1.51215451, 0.83443565]),
                       'table': np.array([0.97949596, 1.06751485, 0.63296875]),
                       'door': np.array([0.53166301, 0.59555772, 1.75001483]),
                       'window': np.array([0.96247056, 0.72462326, 1.14818682]),
                       'bookshelf': np.array([0.83221924, 1.04909355, 1.68756634]),
                       'picture': np.array([0.21132214, 0.4206159 , 0.53728459]),
                       'counter': np.array([1.44400728, 1.89708334, 0.26985747]),
                       'desk': np.array([1.02942616, 1.40407966, 0.87554322]),
                       'curtain': np.array([1.37664116, 0.65521793, 1.68131292]),
                       'refrigerator': np.array([0.66508189, 0.71111926, 1.29885307]),
                       'showercurtain': np.array([0.41999174, 0.37906947, 1.75139715]),
                       'toilet': np.array([0.59359559, 0.59124924, 0.73919014]),
                       'sink': np.array([0.50867595, 0.50656087, 0.30136236]),
                       'bathtub': np.array([1.15115265, 1.0546296 , 0.49706794]),
                       'otherfurniture': np.array([0.47535286, 0.49249493, 0.58021168])
                        }

color_name_to_rgb = {
    'red': [1.0, 0.0, 0.0],
    'green': [0.0, 1.0, 0.23529411764705882],
    'blue': [0.0, 0.4705882352941176, 1.0],
    'purple': [0.7058823529411765, 0.0, 1.0],
    'orange': [1.0, 0.23529411764705882, 0.0],
    'lime': [0.23529411764705882, 1.0, 0.0],
    'cyan': [0.0, 0.7058823529411765, 1.0],
    'magenta': [0.4705882352941176, 0.0, 1.0],
    'pink': [1.0, 0.0, 0.4705882352941176],
    'yellow': [0.4705882352941176, 1.0, 0.0],
    'skyblue': [0.0, 0.9411764705882353, 1.0],
    'indigo': [0.23529411764705882, 0.0, 1.0],
    'hotpink': [1.0, 0.0, 0.7058823529411765],
    'chartreuse': [0.7058823529411765, 1.0, 0.0],
    'aquamarine': [0.0, 1.0, 0.7058823529411765],
    'royalblue': [0.0, 0.23529411764705882, 1.0],
    'violet': [0.9411764705882353, 0.0, 1.0],
    'gold': [1.0, 0.4705882352941176, 0.0],
    'black': [0.0, 0.0, 0.0],
}

class_to_color_name = {
    'bathtub': 'black',
    'bed': 'green',
    'bookshelf': 'blue',
    'cabinet': 'purple',
    'chair': 'orange',
    'counter': 'lime',
    'curtain': 'cyan',
    'desk': 'magenta',
    'door': 'pink',
    'otherfurniture': 'yellow',
    'picture': 'skyblue',
    'refrigerator': 'indigo',
    'showercurtain': 'hotpink',
    'sink': 'chartreuse',
    'sofa': 'aquamarine',
    'table': 'royalblue',
    'toilet': 'violet',
    'window': 'gold',
}
    

def flip_axis_to_camera(pc):
    ''' Flip X-right,Y-forward,Z-up to X-right,Y-down,Z-forward
    Input and output are both (N,3) array
    '''
    pc2 = np.copy(pc)
    pc2[...,[0,1,2]] = pc2[...,[0,2,1]] # cam X,Y,Z = depth X,-Z,Y
    pc2[...,1] *= -1
    return pc2

def flip_axis_to_depth(pc):
    pc2 = np.copy(pc)
    pc2[...,[0,1,2]] = pc2[...,[0,2,1]] # depth X,Y,Z = cam X,Z,-Y
    pc2[...,2] *= -1
    return pc2

# Function to create a bounding box LineSet from center, heading, and size
def create_bbox(center, heading, size, color):
    corners = np.array([
        [-size[0]/2, -size[1]/2, -size[2]/2],
        [ size[0]/2, -size[1]/2, -size[2]/2],
        [ size[0]/2,  size[1]/2, -size[2]/2],
        [-size[0]/2,  size[1]/2, -size[2]/2],
        [-size[0]/2, -size[1]/2,  size[2]/2],
        [ size[0]/2, -size[1]/2,  size[2]/2],
        [ size[0]/2,  size[1]/2,  size[2]/2],
        [-size[0]/2,  size[1]/2,  size[2]/2],
    ])
    
    # Rotate the corners according to the heading
    cos_theta = np.cos(heading)
    sin_theta = np.sin(heading)
    rotation_matrix = np.array([
        [1, 0, 0],
        [0, cos_theta, -sin_theta],
        [0, sin_theta, cos_theta]
    ])
    rotated_corners = np.dot(corners, rotation_matrix.T)
    
    # Translate corners to the center
    corners =  center - rotated_corners
    
    # Create LineSet
    lines = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
        [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
        [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical lines
    ]
    colors = [color for _ in range(len(lines))]  # Red color for all lines
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(corners),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    
    return line_set


# Function to create a bounding box LineSet from corners
def create_bbox_from_corners(corners, color):
    lines = [
        [0, 1], [1, 2], [2, 3], [3, 0],  # Bottom face
        [4, 5], [5, 6], [6, 7], [7, 4],  # Top face
        [0, 4], [1, 5], [2, 6], [3, 7]   # Vertical lines
    ]
    colors = [color for _ in range(len(lines))]  # Same color for all lines
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(corners),
        lines=o3d.utility.Vector2iVector(lines),
    )
    line_set.colors = o3d.utility.Vector3dVector(colors)
    
    return line_set

In [ ]:
from sklearn.neighbors import NearestNeighbors

def plot_pcd_bboxes(img_id):
    
    test_dataset = ScannetValDataset(all_classes=True,
                                    num_novel_class=9,
                                    num_points=40000,
                                    augment=False)
    
    data = test_dataset[img_id] # same as end_points
    
    center_label = data["center_label"]
    heading_class_label = data["heading_class_label"]
    heading_residual_label = data["heading_residual_label"]
    size_residual_label = data["size_residual_label"]
    sem_cls_label = data["sem_cls_label"]
    
    scanname = test_dataset.scan_names[img_id]
    # Get Point Cloud:
    pcd = o3d.io.read_point_cloud(f"../../../../mnt/data/Datasets/ScanNet_v2/scans/{scanname}/{scanname}_vh_clean_2.ply")
    points = np.asarray(pcd.points)
    colors = np.asarray(pcd.colors)
    # print("Points from .ply file: ", points.shape, "/n")    # (60438, 3)
    # print("Colors: ", colors.shape, "/n")   # (60438, 3)
    
    '''
    # Flipping the point clouds:
    pcd_flipped = flip_axis_to_camera(points)
    pcd_flipped = flip_axis_to_depth(pcd_flipped)
    
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(pcd_flipped)
    # point_cloud.colors = o3d.utility.Vector3dVector(colors)
    
    # Create a rotation matrix for rotating around the z-axis
    vector_point_cloud = np.array([0, 1])  # A unit vector along the y-axis
    vector_target = np.array([0, -0.1])  # A vector from the target door coordinates

    # Calculate the angle between the vectors
    angle = np.arctan2(vector_target[1], vector_target[0]) - np.arctan2(vector_point_cloud[1], vector_point_cloud[0])
    cos_theta = np.cos(angle)
    sin_theta = np.sin(angle)

    rotation_matrix = np.array([
        [cos_theta, -sin_theta, 0],
        [sin_theta, cos_theta, 0],
        [0, 0, 1]
    ])
    
    pcd.rotate(rotation_matrix, center=(0, 0, 0))

    # Apply the rotation
    # pcd.rotate(rotation_matrix, center=(0, 0, 0))
    point_cloud = pcd
    '''
    
    # Flipping the point clouds:
    pcd_noRGB = data['point_clouds'] # (40000, 3)
    print("Points from data['point_clouds']: ", np.array(pcd_noRGB).shape)
    pcd_noRGB_flipped = flip_axis_to_camera(pcd_noRGB)
    pcd_noRGB_flipped = flip_axis_to_depth(pcd_noRGB_flipped)
    
    # Nearest Neighbors to map pcd_noRGB_flipped to points
    k = 10
    nn = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(points)
    distances, indices = nn.kneighbors(pcd_noRGB_flipped)
    
    # Calculate the weighted average of the RGB values
    weights = 1 / (distances + 1e-8)  # Avoid division by zero
    weights /= weights.sum(axis=1, keepdims=True)  # Normalize weights
    
    # Compute the weighted average of RGB values
    rgb_vals = np.zeros((pcd_noRGB_flipped.shape[0], 3))
    for i in range(k):
        rgb_vals += colors[indices[:, i]] * weights[:, i:i+1]

    # print("Mapped RGB values shape:", rgb_vals.shape)
    
    point_cloud = o3d.geometry.PointCloud()
    point_cloud.points = o3d.utility.Vector3dVector(pcd_noRGB_flipped)
    point_cloud.colors = o3d.utility.Vector3dVector(rgb_vals)
    
    # point_cloud.paint_uniform_color([0.5, 0.5, 0.5])
    
    # Load the ground truth bounding boxes
    gt_filepath= f"../val_bboxes/{scanname}_gt.npy"
    # print(gt_filepath)
    
    gt = np.load(gt_filepath, allow_pickle=True) # gt bounding boxes for each scene
    
    gt_bounding_boxes = []

    for box_dict in gt:
        bbox = box_dict['bbox']
        gt_bounding_boxes.append(bbox)
        
    print("No. of GT bboxes: ", len(gt_bounding_boxes))
    
    # Plot GT: 
    gt_entities = [point_cloud]
    for i in range(len(gt_bounding_boxes)):
        center = center_label[i]
        heading = heading_class_label[i] * (2 * np.pi / 12) + heading_residual_label[i]
        classname = classes[sem_cls_label[i]]
        size = box_sizes[classname] + size_residual_label[i]
        color = class_to_color_name.get(classname, "red")  # Default to red if class not in color_map
    
        bbox = create_bbox(center, heading, size, color_name_to_rgb[color])
        gt_entities.append(bbox)
        
        print(f"\n Bounding Box {gt[i]['gt_bbox_index']+1}:")
        print(f"Class {classes[gt[i]['class']]}:")
        print(f"Color: {color}")
        print(f"Center: {center}")


    # Visualize the point cloud with bounding boxes using draw_plotly
    print("Plotting point cloud and GT bboxes: \n")
    o3d.visualization.draw_plotly(gt_entities)
    
    
    # Load the predicted bounding boxes
    pred_filepath = f"../val_bboxes/{scanname}_pred.npy"
    print("Loading Pred bboxes from: ", pred_filepath)
    pred = np.load(pred_filepath, allow_pickle=True)
    # print(pred)
    pred_bounding_boxes = []

    for box_dict in pred:
        bbox = box_dict['bbox']
        classname = box_dict['class']
        gt_bbox_idx = box_dict['gt_bbox_index']
        pred_bbox_idx = box_dict['pred_bbox_index']
        
        if bbox is not None:
            pred_bounding_boxes.append((bbox, classname, gt_bbox_idx, pred_bbox_idx))
            
    print("No. of Pred bboxes: ", len(pred_bounding_boxes))
    
    # Plot prediced bboxes: 
    pred_entities = [point_cloud]
    for i in range(len(pred_bounding_boxes)):
        bbox = pred_bounding_boxes[i][0]
        box_size, heading_angle, new_center = inverse_get_3d_box(bbox)
        print(f"bbox {i}: ", bbox)
        classname = classes[pred_bounding_boxes[i][1]]
        gt_bbox_idx = pred_bounding_boxes[i][2]
        pred_bbox_idx = pred_bounding_boxes[i][3]
        
        color = class_to_color_name.get(classname, "red")  # Default to red if class not in color_map
        
        bbox = create_bbox(new_center, heading_angle, box_size, color_name_to_rgb[color])
        pred_entities.append(bbox)
        
        print(f"\n Pred Bounding Box idx: {pred_bbox_idx}")
        print(f"For GT Bounding Box idx: {gt_bbox_idx}")
        print(f"Class {classname}:")
        print(f"Color: {color}")

    print("Length of Pred Entities", len(pred_entities))
    print(pred_entities)
    
    # Visualize the point cloud with bounding boxes using draw_plotly
    print("Plotting point cloud and Pred bboxes: \n")
    o3d.visualization.draw_plotly(pred_entities)

In [ ]:
plot_pcd_bboxes(46)

In [ ]:
plot_pcd_bboxes(1)

In [ ]:
filepath = "../pseudo_bboxes_npy/scene0146_00_pseudo_bboxes.npy"
p_bboxes = np.load(filepath, allow_pickle=True)
print(p_bboxes)
print(p_bboxes.shape)

In [ ]:
def create_bbox(center, size, heading):
    # Calculate corner points of the bounding box
    l, w, h = size
    corners = np.array([
        [-l / 2, -w / 2, -h / 2],
        [l / 2, -w / 2, -h / 2],
        [l / 2, w / 2, -h / 2],
        [-l / 2, w / 2, -h / 2],
        [-l / 2, -w / 2, h / 2],
        [l / 2, -w / 2, h / 2],
        [l / 2, w / 2, h / 2],
        [-l / 2, w / 2, h / 2]
    ])

    # Rotation matrix for heading angle
    cos_theta = np.cos(heading)
    sin_theta = np.sin(heading)
    rotation_matrix = np.array([
        [cos_theta, -sin_theta, 0],
        [sin_theta, cos_theta, 0],
        [0, 0, 1]
    ])

    # Rotate and translate corners
    rotated_corners = np.dot(corners, rotation_matrix.T) + center

    # Define lines for the bounding box edges
    lines = [
        [0, 1], [1, 2], [2, 3], [3, 0],
        [4, 5], [5, 6], [6, 7], [7, 4],
        [0, 4], [1, 5], [2, 6], [3, 7]
    ]

    # Create a LineSet for the bounding box
    line_set = o3d.geometry.LineSet(
        points=o3d.utility.Vector3dVector(rotated_corners),
        lines=o3d.utility.Vector2iVector(lines)
    )
    line_set.colors = o3d.utility.Vector3dVector([[1, 0, 0] for _ in range(len(lines))])

    return line_set

In [ ]:
entities = [pcd]
for bbox in p_bboxes:
    x, y, z, l, w, h, heading, _ = bbox
    center = np.array([x, y, z])
    size = np.array([l, w, h])
    bbox_line_set = create_bbox(center, size, heading)
    entities.append(bbox_line_set)

# Visualize using Open3D
o3d.visualization.draw_plotly(entities)

In [ ]:
filepath = "../pseudo_bboxes_npy/scene0146_00_pseudo_bboxes_0.95.npy"
p_bboxes = np.load(filepath, allow_pickle=True)
print(p_bboxes)
print(p_bboxes.shape)

In [ ]:
entities = [pcd]
for bbox in p_bboxes:
    x, y, z, l, w, h, heading, _ = bbox
    center = np.array([x, y, z])
    size = np.array([l, w, h])
    bbox_line_set = create_bbox(center, size, heading)
    entities.append(bbox_line_set)

# Visualize using Open3D
o3d.visualization.draw_plotly(entities)

# Analyze gt_df results

In [ ]:
import pandas as pd

# Define file paths
gt_df0_filepath = "gt_df_results_seed_0.csv"
gt_df1_filepath = "gt_df_results_seed_1.csv"
gt_df2_filepath = "gt_df_results_seed_2.csv"

# Load DataFrames from CSV files
gt_df0 = pd.read_csv(gt_df0_filepath)
gt_df1 = pd.read_csv(gt_df1_filepath)
gt_df2 = pd.read_csv(gt_df2_filepath)

# Compare values in "has_pred_bbox" column across all DataFrames
# Merge the DataFrames to compare
key_columns = ['img_id', 'scan_name', 'gt_index']

# Ensure the columns for the key exist in all DataFrames
assert all(col in df.columns for col in key_columns for df in [gt_df0, gt_df1, gt_df2]), "Key columns do not exist in all DataFrames."

# Initialize empty DataFrame to store differences
differences_df = pd.DataFrame(columns=['img_id', 'diff_df0_df1', 'diff_df1_df2'])

# Iterate through rows and compare row-wise across DataFrames
for idx, row in gt_df0.iterrows():
    key_values = row[key_columns]
    
    # Locate corresponding rows in gt_df1 and gt_df2 based on the composite key
    row_df1 = gt_df1[(gt_df1[key_columns] == key_values).all(axis=1)]
    row_df2 = gt_df2[(gt_df2[key_columns] == key_values).all(axis=1)]
    
    # Check if rows exist in gt_df1 and gt_df2
    if not row_df1.empty and not row_df2.empty:
        # Compare values in 'has_pred_bbox' column
        if (row['has_pred_bbox'] != row_df1.iloc[0]['has_pred_bbox']) or (row['classname'] != row_df1.iloc[0]['classname']):
            differences_df = differences_df.append({
                'img_id': row['img_id'],
                'diff_df0_df1': f"has_pred_bbox: {row['has_pred_bbox']} != {row_df1.iloc[0]['has_pred_bbox']}, classname: {row['classname']} != {row_df1.iloc[0]['classname']}",
                'diff_df1_df2': f"has_pred_bbox: {row_df1.iloc[0]['has_pred_bbox']} != {row_df2.iloc[0]['has_pred_bbox']}, classname: {row_df1.iloc[0]['classname']} != {row_df2.iloc[0]['classname']}"
            }, ignore_index=True)

    else:
        print(f"Row with key values {key_values} not found in either gt_df1 or gt_df2.")

# Print differences found
if not differences_df.empty:
    print("Differences found in 'has_pred_bbox' column:")
    print(differences_df)
else:
    print("No differences found in 'has_pred_bbox' column.")

In [ ]:
import pandas as pd

# Define file paths
gt_df0_filepath = "gt_df_results_nonseed_0.csv"
gt_df1_filepath = "gt_df_results_nonseed_1.csv"
gt_df2_filepath = "gt_df_results_nonseed_2.csv"

# Load DataFrames from CSV files
gt_df0 = pd.read_csv(gt_df0_filepath)
gt_df1 = pd.read_csv(gt_df1_filepath)
gt_df2 = pd.read_csv(gt_df2_filepath)

# Compare values in "has_pred_bbox" column across all DataFrames
# Merge the DataFrames to compare
key_columns = ['img_id', 'scan_name', 'gt_index']

# Ensure the columns for the key exist in all DataFrames
assert all(col in df.columns for col in key_columns for df in [gt_df0, gt_df1, gt_df2]), "Key columns do not exist in all DataFrames."

# Initialize empty DataFrame to store differences
differences_df = pd.DataFrame(columns=['img_id', 'diff_df0_df1', 'diff_df1_df2'])

# Iterate through rows and compare row-wise across DataFrames
for idx, row in gt_df0.iterrows():
    key_values = row[key_columns]
    
    # Locate corresponding rows in gt_df1 and gt_df2 based on the composite key
    row_df1 = gt_df1[(gt_df1[key_columns] == key_values).all(axis=1)]
    row_df2 = gt_df2[(gt_df2[key_columns] == key_values).all(axis=1)]
    
    # Check if rows exist in gt_df1 and gt_df2
    if not row_df1.empty and not row_df2.empty:
        # Compare values in 'has_pred_bbox' column
        if (row['has_pred_bbox'] != row_df1.iloc[0]['has_pred_bbox']) or (row['classname'] != row_df1.iloc[0]['classname']):
            differences_df = pd.concat([differences_df, pd.DataFrame({
                'img_id': [row['img_id']],
                'diff_df0_df1': [f"has_pred_bbox: {row['has_pred_bbox']} != {row_df1.iloc[0]['has_pred_bbox']}, classname: {row['classname']} != {row_df1.iloc[0]['classname']}"],
                'diff_df1_df2': [f"has_pred_bbox: {row_df1.iloc[0]['has_pred_bbox']} != {row_df2.iloc[0]['has_pred_bbox']}, classname: {row_df1.iloc[0]['classname']} != {row_df2.iloc[0]['classname']}"]
            }, )])

    else:
        print(f"Row with key values {key_values} not found in either gt_df1 or gt_df2.")

# Print differences found
if not differences_df.empty:
    print("Differences found in 'has_pred_bbox' column:")
    print(differences_df)
else:
    print("No differences found in 'has_pred_bbox' column.")